In [5]:
import resnet.models as models

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
import argparse
import numpy as np
import os
import shutil


In [6]:
arch = "resnet"
depth = 56
cuda = torch.cuda.is_available()
seed = 1
save = "./logs"
dataset = "cifar10"
batch_size = 1000
test_batch_size = 1000
kwargs = {'num_workers': 1, 'pin_memory': True} if cuda else {}
lr = 0.1
momentum=0.9
weight_decay=1e-4
log_interval=100
start_epoch = 0
epochs=160

In [7]:
if not os.path.exists(save):
    os.makedirs(save)

In [8]:
if dataset == "cifar10":
    train_loader = torch.utils.data.DataLoader(
        datasets.CIFAR10('./data.cifar10', train=True, download=True,
                       transform=transforms.Compose([
                           transforms.Pad(4),
                           transforms.RandomCrop(32),
                           transforms.RandomHorizontalFlip(),
                           transforms.ToTensor(),
                           transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
                       ])),
        batch_size=batch_size, shuffle=True, **kwargs)
    test_loader = torch.utils.data.DataLoader(
        datasets.CIFAR10('./data.cifar10', train=False, transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
                       ])),
        batch_size=test_batch_size, shuffle=True, **kwargs)

Files already downloaded and verified


In [9]:
model = models.__dict__[arch](dataset=dataset,depth=depth)
# for p in model.parameters():
#     p.requires_grad = True

In [10]:
if cuda:
    model.cuda()

In [11]:
optimizer = optim.SGD(model.parameters(),lr=lr,momentum=momentum,weight_decay=weight_decay)

In [12]:
mini_batch = iter(train_loader)

In [13]:
data,target = next(mini_batch)
data, target = data.cuda(), target.cuda()

In [14]:
data,target = Variable(data),Variable(target)

In [15]:
optimizer.zero_grad()
output = model(data)
loss = F.cross_entropy(output, target)


In [16]:
loss.backward()

In [17]:
grad = [p.grad for p in model.parameters()]


In [18]:
param =[p for p in model.parameters() ]

In [19]:
grad[0][0][0][0][0]

tensor(0.0148, device='cuda:0')

In [20]:
param[0][0][0][0][0]

tensor(-0.1884, device='cuda:0', grad_fn=<SelectBackward>)

In [25]:
for idx,p in enumerate(model.parameters()):
    p.data -= 100 * grad[idx]

In [26]:
param =[p for p in model.parameters() ]

In [27]:
param[0][0][0][0][0]

tensor(-1.6638, device='cuda:0', grad_fn=<SelectBackward>)

In [23]:
l = [10,20,30]

In [24]:
for idx,data in enumerate(l):
    print(idx,data)

0 10
1 20
2 30


In [34]:
model2 = models.__dict__[arch](dataset=dataset,depth=depth)
optimizer2 = optim.SGD(model2.parameters(),lr=lr,momentum=momentum,weight_decay=weight_decay)

In [23]:
optimizer.step()

In [42]:
model2.parameters().
# model.sub_(lr * grad)


<generator object Module.parameters at 0x1553060f05d0>

In [ ]:
def train(epoch):
    # set as train mode
    model.train()
    avg_loss = 0.
    train_acc = 0.
    for batch_idx, (data, target) in enumerate(train_loader):
        if cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        avg_loss += loss.data
        pred = output.data.max(1, keepdim=True)[1]
        train_acc += pred.eq(target.data.view_as(pred)).cpu().sum()
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.1f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data))

In [ ]:
def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        if cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        test_loss += F.cross_entropy(output, target, size_average=False).data # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.1f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    return correct / float(len(test_loader.dataset))


In [ ]:
def save_checkpoint(state, is_best, filepath):
    torch.save(state, os.path.join(filepath, 'checkpoint.pth.tar'))
    if is_best:
        shutil.copyfile(os.path.join(filepath, 'checkpoint.pth.tar'), os.path.join(filepath, 'model_best.pth.tar'))

In [ ]:
best_prec1 = 0.
for epoch in range(start_epoch, epochs):
    if epoch in [epochs*0.5, epochs*0.75]:
        for param_group in optimizer.param_groups:
            param_group['lr'] *= 0.1
    train(epoch)
    prec1 = test()
    is_best = prec1 > best_prec1
    best_prec1 = max(prec1, best_prec1)
    save_checkpoint({
        'epoch': epoch + 1,
        'state_dict': model.state_dict(),
        'best_prec1': best_prec1,
        'optimizer': optimizer.state_dict(),
        'cfg': model.cfg
    }, is_best, filepath=save)